In [1]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tools
import geopandas as gpd
from shapely.geometry import LineString
import sys
import os
sys.path.append('/Applications/WhiteboxTools_darwin_m_series/WBT')  # Path to whitebox_tools.py
from whitebox_tools import WhiteboxTools
wbt = WhiteboxTools()

In [2]:
buffer_distance = 0.5  # adjust to half your raster resolution or desired width (in map units!)

cleaned_dem_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'
streams_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp'
out_fp_streams = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/PROCESSED/5haStreams_0.5m_buf{buffer_distance}.tif'
ditches_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp'
out_fp_ditches = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m_buf{buffer_distance}.tif'

# Streams
streams = gpd.read_file(streams_fp)
streams_buffered = streams.copy()
streams_buffered["geometry"] = streams.geometry.buffer(buffer_distance)

streams_buffered_fp = streams_fp.replace(".shp", f"_buf{buffer_distance}m.shp")
streams_buffered.to_file(streams_buffered_fp)

# Ditches
ditches = gpd.read_file(ditches_fp)
ditches_buffered = ditches.copy()
ditches_buffered["geometry"] = ditches.geometry.buffer(buffer_distance)

ditches_buffered_fp = ditches_fp.replace(".shp", f"_buf{buffer_distance}m.shp")
ditches_buffered.to_file(ditches_buffered_fp)

tools.rasterize_shapefile(
    shapefile=streams_buffered_fp,
    burn_field='GRID_CODE',
    subset=None,
    out_fp=out_fp_streams,
    ref_raster=cleaned_dem_fp,
    plot=False,
    save_in='geotiff'
)

tools.rasterize_shapefile(
    shapefile=ditches_buffered_fp,
    burn_field='VALUE',
    subset=None,
    out_fp=out_fp_ditches,
    ref_raster=cleaned_dem_fp,
    plot=False,
    save_in='geotiff'
)

for file in [out_fp_ditches, out_fp_streams]:
    if file == out_fp_ditches:
        file_end = 'channels'
    elif file == out_fp_streams:
        file_end = '5haStreams'        
    src_fp = file
    binary_fp = src_fp.replace(".tif", "_binary.tif")

    with rasterio.open(src_fp) as src:
        arr = src.read(1)
        profile = src.profile
    
        # Create binary: 0 = stream/ditch, 1 = background
        binary = np.where(arr > 0, 1, 0).astype(np.uint8)
    
        profile.update(dtype=rasterio.uint8, nodata=255)
    
        with rasterio.open(binary_fp, "w", **profile) as dst:
            dst.write(binary, 1)
    
    # Now run Euclidean distance on the binary raster
    output = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/PROCESSED/{file_end}_distances.tif'
    
    wbt.euclidean_distance(
        i=binary_fp,
        output=output
    )

/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field created_da create as date field, though DateTime requested.
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field last_edite create as date field, though DateTime requested.
  ogr_write(


./whitebox_tools --run="EuclideanDistance" --input='/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5_0.5m_buf0.5_binary.tif' --output='/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/PROCESSED/channels_distances.tif' -v --compress_rasters=False

********************************
* Welcome to EuclideanDistance *
* Powered by WhiteboxTools     *
* www.whiteboxgeo.com          *
********************************
Reading data...
Initializing Rasters: 0%
Initializing Rasters: 1%
Initializing Rasters: 2%
Initializing Rasters: 3%
Initializing Rasters: 4%
Initializing Rasters: 5%
Initializing Rasters: 6%
Initializing Rasters: 7%
Initializing Rasters: 8%
Initializing Rasters: 9%
Initializing Rasters: 10%
Initializing Rasters: 11%
Initializing Rasters: 12%
Initializing Rasters: 13%
Initializing Rasters: 14%
Initializing Rasters: 15%
In

In [3]:
# Precomputed full-resolution distance raster
file_ends = ['channels', '5haStreams']

for file in file_ends:
    dist_05m_fp = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/PROCESSED/{file}_distances.tif'
    resos = [20, 40, 80, 160]
    catchment_no = np.arange(1, 23)
    exclude = [11, 17, 18, 19]
    catchs = np.setdiff1d(catchment_no, exclude)
    
    nodata_val = -9999  # value for NoData in output
    
    for catch in catchs:
        for reso in resos:
            outdir = f"/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}"
            os.makedirs(outdir, exist_ok=True)
    
            # Reference raster for resolution, extent, and mask
            ref_raster_fp = f"{outdir}/{file}.asc"
            cmask_fp = f"{outdir}/cmask.asc"
    
            # Resample Euclidean distance to reference raster
            reproj_fp = os.path.join(outdir, f"{file}_distance.asc")
            tools.reproj_match(
                infile=dist_05m_fp,
                match=ref_raster_fp,
                outfile=reproj_fp,
                resampling_method='average',
                save_in='asc')
    
            # -----------------------------
            # Apply cmask masking and save as a new file
            # -----------------------------
            #masked_fp = os.path.join(outdir, "channel_distance_masked.asc")
            with rasterio.open(reproj_fp) as dist_src:
                dist_data = dist_src.read(1)
                meta = dist_src.meta.copy()
    
                with rasterio.open(ref_raster_fp) as mask_src:
                    mask_data = mask_src.read(1)
                    mask_nodata = mask_src.nodata
    
                # Mask: wherever cmask == nodata, set distance raster to -9999
                if mask_nodata is not None:
                    dist_data[mask_data == mask_nodata] = nodata_val
    
                # Update metadata for new file
                meta.update(nodata=nodata_val)
    
                # Save masked raster as a new file
                with rasterio.open(reproj_fp, 'w', **meta) as dst:
                    dst.write(dist_data.astype(np.float32), 1)

Coregistered to shape: 69 41 
 Affine | 20.00, 0.00, 732306.00|
| 0.00,-20.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 35 21 
 Affine | 40.00, 0.00, 732286.00|
| 0.00,-40.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 18 11 
 Affine | 80.00, 0.00, 732246.00|
| 0.00,-80.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 9 6 
 Affine | 160.00, 0.00, 732166.00|
| 0.00,-160.00, 7134870.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 39 30 
 Affine | 20.00, 0.00, 730786.00|
| 0.00,-20.00, 7134570.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 20 16 
 Affine | 40.00, 0.00, 730766.00|
| 0.00,-40.00, 7134590.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 11 9 
 Affine | 80.00, 0.00, 730726.00|
| 0.00,-80.00, 7134630.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 6 5 
 Affine | 160.00, 0.00, 730726.00|
| 0.00,-160.00, 7134710.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 13 21 
 Affine | 20.00, 0.00, 731326.00|
| 0.00,-20.00, 7134490.00|
| 0.00, 0.00, 1.00|
Co

In [1]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from shapely.geometry import box

# -----------------------------
# User settings
# -----------------------------
shapefiles = [
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_5ha_streams/5haStreams.shp',
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/ManualStreams/PROCESSED/5haStreams_ManuallyDigitizedDitchesKrycklan_merged_5.shp'
]
file_ends = ['5haStreams', 'channels']

resos = [20, 40, 80, 160]
catchment_no = np.arange(1, 23)
exclude = [11, 17, 18, 19]
catchs = np.setdiff1d(catchment_no, exclude)

nodata_val = -9999

# -----------------------------
# Main loop
# -----------------------------
for shp_fp, file_end in zip(shapefiles, file_ends):
    # Load shapefile once and reproject to raster CRS
    gdf = gpd.read_file(shp_fp)
    gdf = gdf.to_crs(epsg=3006)  # SWEREF99 TM

    for catch in catchs:
        for reso in resos:
            outdir = f"/Users/jpnousu/Krycklan_GIS_data/{reso}m/C{catch}"
            os.makedirs(outdir, exist_ok=True)

            ref_raster_fp = f"{outdir}/{file_end}.asc"
            cmask_fp = f"{outdir}/cmask.asc"

            if not os.path.exists(ref_raster_fp):
                print(f"⚠️ Missing reference raster {ref_raster_fp}")
                continue
            if not os.path.exists(cmask_fp):
                print(f"⚠️ Missing cmask raster {cmask_fp}")
                continue

            # Open reference raster for alignment
            with rasterio.open(ref_raster_fp) as ref:
                profile = ref.profile.copy()
                profile.update(dtype='float32', nodata=nodata_val)
                transform = ref.transform
                width, height = ref.width, ref.height
                bounds = ref.bounds

            # Clip lines to raster extent
            bbox = box(*bounds)
            gdf_clip = gdf.clip(bbox)

            # Initialize output array
            arr = np.full((height, width), nodata_val, dtype='float32')

            if not gdf_clip.empty:
                # Loop through raster grid
                for row in range(height):
                    for col in range(width):
                        # Compute cell bounds
                        x_min, y_max = transform * (col, row)
                        x_max = x_min + transform.a
                        y_min = y_max + transform.e  # transform.e is negative
                        cell_poly = box(x_min, y_min, x_max, y_max)

                        # Intersect with clipped lines
                        inter = gdf_clip.intersection(cell_poly)

                        # Sum the lengths of intersected portions
                        total_length = sum(geom.length for geom in inter if not geom.is_empty)

                        if total_length > 0:
                            arr[row, col] = total_length

            # Apply channel mask: set cells outside channels to nodata
            with rasterio.open(ref_raster_fp) as mask_src:
                mask_data = mask_src.read(1)
                mask_nodata = mask_src.nodata

            arr[mask_data == mask_nodata] = nodata_val

            # Save masked length raster
            out_fp = f"{outdir}/{file_end}_length.asc"
            with rasterio.open(out_fp, 'w', **profile) as dst:
                dst.write(arr, 1)

            print(f"✅ Saved masked length raster: {out_fp}")

✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/20m/C1/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/40m/C1/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/80m/C1/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/160m/C1/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/20m/C2/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/40m/C2/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/80m/C2/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/160m/C2/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/20m/C3/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/40m/C3/5haStreams_length.asc
✅ Saved masked length raster: /Users/jpnousu/Krycklan_GIS_data/80m/C3/5haStrea